In [1]:
import json

with open('data/points.json', 'r') as json_file:
    points = json.load(json_file)

with open('data/users.json', 'r') as json_file:
    users = json.load(json_file)

with open('data/activity_logs.json', 'r') as json_file:
    activity_logs = json.load(json_file)

In [2]:
import polars as pl

## Point DF

In [3]:
points_df = pl.DataFrame(list(points.values()))
points_df = points_df.drop("description")
points_df = points_df.select([pl.col('*').exclude('location'), pl.col('location').struct.rename_fields(["long", "lat"])]).unnest("location")
points_df

point,isPublic,heroImage,long,lat
str,bool,str,f64,f64
"""天水圍體育館""",true,"""https://firebasestorage.google…",114.00674,22.454822
"""隱藏集合點""",true,"""""",114.008563,22.451097
"""流浮山消防局""",true,"""https://firebasestorage.google…",114.002262,22.471316
"""天水圍運動場""",true,"""https://firebasestorage.google…",114.003714,22.453907
"""慧景軒巴士站""",true,"""https://firebasestorage.google…",114.005047,22.465724
…,…,…,…,…
"""聖葉理諾堂""",true,"""https://firebasestorage.google…",114.001623,22.44961
"""天瑞俓（一）""",true,"""https://firebasestorage.google…",113.998795,22.449919
"""天河路遊樂場涼亭""",true,"""https://firebasestorage.google…",114.000367,22.449815


## Users DF

In [4]:
from datetime import datetime

def map_score(data):
    score_per_second = {
        "_": 1 / 60,
        "1": 1 / 60,
        "2": 20 / 60,
        "3": 150 / 60,
        "4": 250 / 60,
        "5": 500 / 60,
    }
    
    total_score = 0
    for item in data:
        created_at = datetime.fromisoformat(item['createdAt'].replace('Z', '+00:00'))
        expired_at = datetime.fromisoformat(item['expiredAt'].replace('Z', '+00:00'))
        duration = (expired_at - created_at).total_seconds()
        level = str(item['level'])
        score = duration * score_per_second.get(level, score_per_second["_"])
        total_score += score
    return total_score

def map_duration(data):
    total_duration = 0
    for item in data:
        created_at = datetime.fromisoformat(item['createdAt'].replace('Z', '+00:00'))
        expired_at = datetime.fromisoformat(item['expiredAt'].replace('Z', '+00:00'))
        duration = (expired_at - created_at).total_seconds()
        total_duration += duration
        
    return total_duration

def map_average_duration(data):
    total_duration = 0
    for item in data:
        created_at = datetime.fromisoformat(item['createdAt'].replace('Z', '+00:00'))
        expired_at = datetime.fromisoformat(item['expiredAt'].replace('Z', '+00:00'))
        duration = (expired_at - created_at).total_seconds()
        total_duration += duration
        
    len = data.len() if data.len() > 0 else 1
    return total_duration / len

def map_total_captured_unique(data):
    pointsId = [x['pointId'] for x in data]
        
    return len(set(pointsId))


def map_final_level(data):
    max_level = 1
    
    for item in data:
        if(item['level'] > max_level):
            max_level = item['level']
    return max_level

In [5]:
users_df = pl.DataFrame(list(users.values()))
users_df = users_df.with_columns(pl.col('capturedPoints').list.len().alias('total_captured'))
users_df = users_df.with_columns(pl.col('capturedPoints').map_elements(map_total_captured_unique, return_dtype=pl.Int8).alias('total_captured_unique'))
users_df = users_df.with_columns(pl.col('capturedPoints').map_elements(map_score, return_dtype=pl.Float32).alias('score'))
users_df = users_df.with_columns(pl.col('score').alias('score_offset') - 12500)
users_df = users_df.with_columns(pl.col('capturedPoints').map_elements(map_duration, return_dtype=pl.Float32).alias('duration'))
users_df = users_df.with_columns(pl.col('capturedPoints').map_elements(map_average_duration, return_dtype=pl.Float32).alias('average_duration'))
users_df = users_df.with_columns(pl.col('capturedPoints').map_elements(map_final_level, return_dtype=pl.Int8).alias('final_level')).drop('level')
users_df = users_df.sort('score', descending=True)
users_df

capturedPoints,email,upgradedPoints,name,total_captured,total_captured_unique,score,score_offset,duration,average_duration,final_level
list[struct[7]],str,list[str],str,u32,i8,f32,f32,f32,f32,i8
"[{""skc-3@example.com"",""南葵中心3隊"",""2024-08-02T21:44:31.341Z"",""2024-08-02T20:27:48.160Z"",2,""MM12lHGWCtvCbQvMy29H"",""天瑞俓（二）""}, {""skc-3@example.com"",""南葵中心3隊"",""2024-08-02T21:42:11.260Z"",""2024-08-02T20:29:51.988Z"",2,""Y54hD80CffPsd7hG6KZn"",""天瑞俓（三）""}, … {""skc-3@example.com"",""南葵中心3隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T21:48:30.321Z"",3,""xv6F5QsTKUVQasrDM5V5"",""天瑞俓（一）""}]","""skc-3@example.com""","[""1"", ""3""]","""南葵中心3隊""",19,10,55591.414062,43091.414062,46675.914062,2456.626953,3
"[{""skc-4@example.com"",""南葵中心4隊"",""2024-08-02T20:22:21.223Z"",""2024-08-02T20:19:06.128Z"",1,""xFldNrcu765eRZc2hrsn"",""聖葉理諾堂""}, {""skc-4@example.com"",""南葵中心4隊"",""2024-08-02T21:41:43.053Z"",""2024-08-02T20:20:21.384Z"",1,""yhDNSAU3ByPksCfmW3E3"",""天河路遊樂場涼亭""}, … {""skc-4@example.com"",""南葵中心4隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T21:42:41.441Z"",3,""gFJqiYep8W1OwbFTueIM"",""天瑞路公園涼亭""}]","""skc-4@example.com""","[""4"", ""1""]","""南葵中心4隊""",26,10,44894.796875,32394.796875,77038.289062,2963.01123,3
"[{""sw-2@example.com"",""上環/將軍澳中心隊"",""2024-08-02T21:46:58.326Z"",""2024-08-02T20:23:05.036Z"",1,""0gHiSX9dMHkVpffFR9RU"",""天水圍體育館""}, {""sw-2@example.com"",""上環/將軍澳中心隊"",""2024-08-02T21:41:47.810Z"",""2024-08-02T20:27:41.262Z"",1,""Un4smPc0lSA8jpfvA5zJ"",""天水圍游泳池""}, … {""sw-2@example.com"",""上環/將軍澳中心隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T22:15:26.468Z"",3,""rSnhF4HYEnTG98p9E7To"",""天水圍公園（西）""}]","""sw-2@example.com""","[""2"", ""4""]","""上環/將軍澳中心隊""",12,12,15474.444336,2974.444336,44609.160156,3717.429932,3
"[{""skc-2@example.com"",""南葵中心2隊"",""2024-08-02T20:35:10.285Z"",""2024-08-02T20:33:36.576Z"",1,""45ntnYbmnHBUIEyvTtNs"",""天瑞社區中心""}, {""skc-2@example.com"",""南葵中心2隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T20:51:29.331Z"",2,""stpbAJ0ixmFg8KELxSzz"",""天榮路西""}, … {""skc-2@example.com"",""南葵中心2隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T21:56:04.014Z"",2,""2d52tp09iWfccGggsMH5"",""流浮山消防局""}]","""skc-2@example.com""","[""4""]","""南葵中心2隊""",15,15,15448.828125,2948.828125,46435.507812,3095.700439,2
"[{""skc-1@example.com"",""南葵中心1隊"",""2024-08-02T21:07:15.858Z"",""2024-08-02T20:22:21.223Z"",1,""xFldNrcu765eRZc2hrsn"",""聖葉理諾堂""}, {""skc-1@example.com"",""南葵中心1隊"",""2024-08-02T21:41:43.053Z"",""2024-08-02T20:24:01.956Z"",1,""yhDNSAU3ByPksCfmW3E3"",""天河路遊樂場涼亭""}, … {""skc-1@example.com"",""南葵中心1隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T22:06:50.843Z"",2,""1ZflGseWvmvEMDtj6YOq"",""隱藏集合點""}]","""skc-1@example.com""","[""4""]","""南葵中心1隊""",23,17,10107.545898,-2392.454102,47757.152344,2076.397949,2
"[{""sd1-1@example.com"",""南區中心1隊"",""2024-08-02T22:35:06.177Z"",""2024-08-02T22:03:00.057Z"",3,""Un4smPc0lSA8jpfvA5zJ"",""天水圍游泳池""}, {""sd1-1@example.com"",""南區中心1隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T22:06:50.535Z"",3,""0gHiSX9dMHkVpffFR9RU"",""天水圍體育館""}]","""sd1-1@example.com""","[""1"", ""2""]","""南區中心1隊""",2,2,8818.417969,-3681.582031,3527.366943,1763.683472,3
"[{""sw-1@example.com"",""上環中心隊"",""2024-08-02T22:35:06.177Z"",""2024-08-02T21:41:47.810Z"",2,""Un4smPc0lSA8jpfvA5zJ"",""天水圍游泳池""}, {""sw-1@example.com"",""上環中心隊"",""2024-08-02T22:00:21.126Z"",""2024-08-02T21:46:58.326Z"",2,""0gHiSX9dMHkVpffFR9RU"",""天水圍體育館""}, … {""sw-1@example.com"",""上環中心隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T22:37:37.693Z"",3,""0gHiSX9dMHkVpffFR9RU"",""天水圍體育館""}]","""sw-1@example.com""","[""1"", ""2""]","""上環中心隊""",7,5,6006.617188,-6493.382812,5870.325195,838.617859,3
"[{""sy1@example.com"",""石蔭中心隊"",""2024-08-02T20:23:05.036Z"",""2024-08-02T20:14:55.939Z"",1,""0gHiSX9dMHkVpffFR9RU"",""天水圍體育館""}, {""sy1@example.com"",""石蔭中心隊"",""2024-08-02T20:27:41.262Z"",""2024-08-02T20:18:47.636Z"",1,""Un4smPc0lSA8jpfvA5zJ"",""天水圍游泳池""}, … {""sy1@example.com"",""石蔭中心隊"",""2024-08-02T21:40:56.120Z"",""2024-08-02T21:35:33.895Z"",3,""45ntnYbmnH

In [6]:
activity_logs_items = list(activity_logs.values())

In [7]:
captured_logs = [x for x in activity_logs_items if x['type'] == 'CAPTURE_POINT']
captured_logs_df = pl.DataFrame(captured_logs)

captured_logs_df = captured_logs_df.with_columns(pl.col('user').map_elements(lambda x: x['email'], return_dtype=pl.String).alias('user_email'))
captured_logs_df = captured_logs_df.with_columns(pl.col('user').map_elements(lambda x: x['name'], return_dtype=pl.String).alias('user_name'))
captured_logs_df = captured_logs_df.with_columns(pl.col('user').map_elements(lambda x: x['level'], return_dtype=pl.Int8).alias('user_level'))
captured_logs_df = captured_logs_df.drop('user')

captured_logs_df = captured_logs_df.with_columns(pl.col('point').map_elements(lambda x: x['pointId'], return_dtype=pl.String).alias('point_id'))
captured_logs_df = captured_logs_df.with_columns(pl.col('point').map_elements(lambda x: x['pointName'], return_dtype=pl.String).alias('point_name'))
captured_logs_df = captured_logs_df.with_columns(pl.col('point').map_elements(lambda x: x['userId'], return_dtype=pl.String).alias('point_user_email'))
captured_logs_df = captured_logs_df.with_columns(pl.col('point').map_elements(lambda x: x['userName'], return_dtype=pl.String).alias('point_user_name'))
captured_logs_df = captured_logs_df.drop('point')

captured_logs_df = captured_logs_df.with_columns(pl.col('datetime').str.to_datetime().dt.replace_time_zone("Asia/Hong_Kong"))

captured_logs_df = captured_logs_df.sort('datetime', descending=True)

captured_logs_df

datetime,type,user_email,user_name,user_level,point_id,point_name,point_user_email,point_user_name
"datetime[μs, Asia/Hong_Kong]",str,str,str,i8,str,str,str,str
2024-08-02 22:37:38.056 HKT,"""CAPTURE_POINT""","""sw-1@example.com""","""上環中心隊""",3,"""0gHiSX9dMHkVpffFR9RU""","""天水圍體育館""","""sw-1@example.com""","""上環中心隊"""
2024-08-02 22:35:06.707 HKT,"""CAPTURE_POINT""","""sw-1@example.com""","""上環中心隊""",3,"""Un4smPc0lSA8jpfvA5zJ""","""天水圍游泳池""","""sw-1@example.com""","""上環中心隊"""
2024-08-02 22:26:56.297 HKT,"""CAPTURE_POINT""","""sw-1@example.com""","""上環中心隊""",3,"""YvzEAhiBhJrOstmGXoH3""","""天榮路東""","""sw-1@example.com""","""上環中心隊"""
2024-08-02 22:22:50.933 HKT,"""CAPTURE_POINT""","""sw-1@example.com""","""上環中心隊""",3,"""IeOJr7CsyHXad2DUL3n4""","""栢慧豪廷門口""","""sw-1@example.com""","""上環中心隊"""
2024-08-02 22:15:27.703 HKT,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",3,"""rSnhF4HYEnTG98p9E7To""","""天水圍公園（西）""","""sw-2@example.com""","""上環/將軍澳中心隊"""
…,…,…,…,…,…,…,…,…
2024-08-02 20:22:29.130 HKT,"""CAPTURE_POINT""","""skc-1@example.com""","""南葵中心1隊""",1,"""xFldNrcu765eRZc2hrsn""","""聖葉理諾堂""","""skc-1@example.com""","""南葵中心1隊"""
2024-08-02 20:20:21.639 HKT,"""CAPTURE_POINT""","""skc-4@example.com""","""南葵中心4隊""",1,"""yhDNSAU3ByPksCfmW3E3""","""天河路遊樂場涼亭""","""skc-4@example.com""","""南葵中心4隊"""
2024-08-02 20:19:06.398 HKT,"""CAPTURE_POINT""","""skc-4@example.com""","""南葵中心4隊""",1,"""xFldNrcu765eRZc2hrsn""","""聖葉理諾堂""","""skc-4@example.com""","""南葵中心4隊"""


In [8]:
clear_logs = [x for x in activity_logs_items if x['type'] == 'CLEAR_POINT']
clear_logs_df = pl.DataFrame(clear_logs)

clear_logs_df = clear_logs_df.with_columns(pl.col('user').map_elements(lambda x: x['email'], return_dtype=pl.String).alias('user_email'))
clear_logs_df = clear_logs_df.with_columns(pl.col('user').map_elements(lambda x: x['name'], return_dtype=pl.String).alias('user_name'))
clear_logs_df = clear_logs_df.drop('user')

clear_logs_df = clear_logs_df.with_columns(pl.col('point').map_elements(lambda x: x['pointId'], return_dtype=pl.String).alias('point_id'))
clear_logs_df = clear_logs_df.with_columns(pl.col('point').map_elements(lambda x: x['pointName'], return_dtype=pl.String).alias('point_name'))
clear_logs_df = clear_logs_df.with_columns(pl.col('point').map_elements(lambda x: x['userId'], return_dtype=pl.String).alias('point_user_email'))
clear_logs_df = clear_logs_df.with_columns(pl.col('point').map_elements(lambda x: x['userName'], return_dtype=pl.String).alias('point_user_name'))
clear_logs_df = clear_logs_df.drop('point')

clear_logs_df = clear_logs_df.with_columns(pl.col('datetime').str.to_datetime().dt.replace_time_zone("Asia/Hong_Kong"))

clear_logs_df = clear_logs_df.sort('datetime', descending=True)

clear_logs_df

datetime,type,user_email,user_name,point_id,point_name,point_user_email,point_user_name
"datetime[μs, Asia/Hong_Kong]",str,str,str,str,str,str,str
2024-08-02 22:35:06.501 HKT,"""CLEAR_POINT""","""sw-1@example.com""","""上環中心隊""","""Un4smPc0lSA8jpfvA5zJ""","""天水圍游泳池""","""sd1-1@example.com""","""南區中心1隊"""
2024-08-02 22:26:56.108 HKT,"""CLEAR_POINT""","""sw-1@example.com""","""上環中心隊""","""YvzEAhiBhJrOstmGXoH3""","""天榮路東""","""skc-1@example.com""","""南葵中心1隊"""
2024-08-02 22:22:50.758 HKT,"""CLEAR_POINT""","""sw-1@example.com""","""上環中心隊""","""IeOJr7CsyHXad2DUL3n4""","""栢慧豪廷門口""","""skc-2@example.com""","""南葵中心2隊"""
2024-08-02 22:15:27.155 HKT,"""CLEAR_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""","""rSnhF4HYEnTG98p9E7To""","""天水圍公園（西）""","""skc-4@example.com""","""南葵中心4隊"""
2024-08-02 22:13:59.970 HKT,"""CLEAR_POINT""","""sw-1@example.com""","""上環中心隊""","""778jzmzKpOvgRHIHcHms""","""麗湖居""","""skc-1@example.com""","""南葵中心1隊"""
…,…,…,…,…,…,…,…
2024-08-02 20:27:41.528 HKT,"""CLEAR_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""","""Un4smPc0lSA8jpfvA5zJ""","""天水圍游泳池""","""sy1@example.com""","""石蔭中心隊"""
2024-08-02 20:25:48.553 HKT,"""CLEAR_POINT""","""skc-1@example.com""","""南葵中心1隊""","""MM12lHGWCtvCbQvMy29H""","""天瑞俓（二）""","""skc-4@example.com""","""南葵中心4隊"""
2024-08-02 20:24:07.066 HKT,"""CLEAR_POINT""","""skc-1@example.com""","""南葵中心1隊""","""yhDNSAU3ByPksCfmW3E3""","""天河路遊樂場涼亭""","""skc-4@example.com""","""南葵中心4隊"""


In [9]:
points_logs = pl.concat([captured_logs_df, clear_logs_df], how='align')

points_logs

datetime,type,user_email,user_name,user_level,point_id,point_name,point_user_email,point_user_name
"datetime[μs, Asia/Hong_Kong]",str,str,str,i8,str,str,str,str
2024-08-02 20:14:56.154 HKT,"""CAPTURE_POINT""","""sy1@example.com""","""石蔭中心隊""",1,"""0gHiSX9dMHkVpffFR9RU""","""天水圍體育館""","""sy1@example.com""","""石蔭中心隊"""
2024-08-02 20:18:47.907 HKT,"""CAPTURE_POINT""","""sy1@example.com""","""石蔭中心隊""",1,"""Un4smPc0lSA8jpfvA5zJ""","""天水圍游泳池""","""sy1@example.com""","""石蔭中心隊"""
2024-08-02 20:19:06.398 HKT,"""CAPTURE_POINT""","""skc-4@example.com""","""南葵中心4隊""",1,"""xFldNrcu765eRZc2hrsn""","""聖葉理諾堂""","""skc-4@example.com""","""南葵中心4隊"""
2024-08-02 20:20:21.639 HKT,"""CAPTURE_POINT""","""skc-4@example.com""","""南葵中心4隊""",1,"""yhDNSAU3ByPksCfmW3E3""","""天河路遊樂場涼亭""","""skc-4@example.com""","""南葵中心4隊"""
2024-08-02 20:22:24.971 HKT,"""CLEAR_POINT""","""skc-1@example.com""","""南葵中心1隊""",null,"""xFldNrcu765eRZc2hrsn""","""聖葉理諾堂""","""skc-4@example.com""","""南葵中心4隊"""
…,…,…,…,…,…,…,…,…
2024-08-02 22:26:56.108 HKT,"""CLEAR_POINT""","""sw-1@example.com""","""上環中心隊""",null,"""YvzEAhiBhJrOstmGXoH3""","""天榮路東""","""skc-1@example.com""","""南葵中心1隊"""
2024-08-02 22:26:56.297 HKT,"""CAPTURE_POINT""","""sw-1@example.com""","""上環中心隊""",3,"""YvzEAhiBhJrOstmGXoH3""","""天榮路東""","""sw-1@example.com""","""上環中心隊"""
2024-08-02 22:35:06.501 HKT,"""CLEAR_POINT""","""sw-1@example.com""","""上環中心隊""",null,"""Un4smPc0lSA8jpfvA5zJ""","""天水圍游泳池""","""sd1-1@example.com""","""南區中心1隊"""


In [10]:
upgrade_logs = [x for x in activity_logs_items if x['type'] == 'USER_UPGRADE']
upgrade_logs_df = pl.DataFrame(upgrade_logs)

upgrade_point = {
    "1": "屏欣苑",
    "2": "銀座",
    "3": "流浮山消防局",
    "4": "天瑞街市",
}

upgrade_logs_df = upgrade_logs_df.with_columns(pl.col('user').map_elements(lambda x: x['email'], return_dtype=pl.String).alias('user_email'))
upgrade_logs_df = upgrade_logs_df.with_columns(pl.col('user').map_elements(lambda x: x['name'], return_dtype=pl.String).alias('user_name'))
upgrade_logs_df = upgrade_logs_df.with_columns(pl.col('point').replace_strict(upgrade_point).alias('upgrade_point')).drop('point')
upgrade_logs_df = upgrade_logs_df.drop('user')

upgrade_logs_df = upgrade_logs_df.with_columns(pl.col('datetime').str.to_datetime().dt.replace_time_zone("Asia/Hong_Kong"))

upgrade_logs_df

datetime,type,user_email,user_name,upgrade_point
"datetime[μs, Asia/Hong_Kong]",str,str,str,str
2024-08-02 21:16:40.334 HKT,"""USER_UPGRADE""","""skc-3@example.com""","""南葵中心3隊""","""流浮山消防局"""
2024-08-02 21:30:30.259 HKT,"""USER_UPGRADE""","""sy1@example.com""","""石蔭中心隊""","""天瑞街市"""
2024-08-02 22:07:56.676 HKT,"""USER_UPGRADE""","""sw-1@example.com""","""上環中心隊""","""銀座"""
2024-08-02 20:57:03.568 HKT,"""USER_UPGRADE""","""sy1@example.com""","""石蔭中心隊""","""流浮山消防局"""
2024-08-02 21:48:34.104 HKT,"""USER_UPGRADE""","""sd1-1@example.com""","""南區中心1隊""","""銀座"""
…,…,…,…,…
2024-08-02 20:42:26.963 HKT,"""USER_UPGRADE""","""skc-1@example.com""","""南葵中心1隊""","""天瑞街市"""
2024-08-02 21:39:58.737 HKT,"""USER_UPGRADE""","""skc-4@example.com""","""南葵中心4隊""","""屏欣苑"""
2024-08-02 20:16:51.826 HKT,"""USER_UPGRADE""","""skc-3@example.com""","""南葵中心3隊""","""屏欣苑"""


In [11]:
patrol_logs = [x for x in activity_logs_items if x['type'] == 'PATROL_CLEAR_POINT'][0]
patrol_logs_df = pl.DataFrame(patrol_logs)

patrol_logs_df = patrol_logs_df.with_columns(pl.col('datetime').str.to_datetime().dt.replace_time_zone("Asia/Hong_Kong"))

patrol_logs_df

point,datetime,type
struct[7],"datetime[μs, Asia/Hong_Kong]",str
"{""sy1@example.com"",""石蔭中心隊"",""2024-08-02T22:33:31.782000"",""2024-08-02T20:45:43.903000"",1,""pbrw5fgRV2MRa5TDj7VB"",""天葵天橋""}",2024-08-02 22:33:32.280 HKT,"""PATROL_CLEAR_POINT"""


## Analysis

### Team Ranking

In [12]:
import plotly.express as px

color_discrete_map = {
    "南葵中心3隊": "#ea3323",
    "南葵中心4隊": "#ff8b00",
    "上環/將軍澳中心隊": "#febb26",
    "南葵中心2隊": "#17873f",
    "南葵中心1隊": "#25dc67",
    "南區中心1隊": "#4f7af9",
    "上環中心隊": "#9c78fe",
    "石蔭中心隊": "#ffccdd",
    "南區中心2隊": "#ffffff",
}

In [13]:
users_df.head(2)

capturedPoints,email,upgradedPoints,name,total_captured,total_captured_unique,score,score_offset,duration,average_duration,final_level
list[struct[7]],str,list[str],str,u32,i8,f32,f32,f32,f32,i8
"[{""skc-3@example.com"",""南葵中心3隊"",""2024-08-02T21:44:31.341Z"",""2024-08-02T20:27:48.160Z"",2,""MM12lHGWCtvCbQvMy29H"",""天瑞俓（二）""}, {""skc-3@example.com"",""南葵中心3隊"",""2024-08-02T21:42:11.260Z"",""2024-08-02T20:29:51.988Z"",2,""Y54hD80CffPsd7hG6KZn"",""天瑞俓（三）""}, … {""skc-3@example.com"",""南葵中心3隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T21:48:30.321Z"",3,""xv6F5QsTKUVQasrDM5V5"",""天瑞俓（一）""}]","""skc-3@example.com""","[""1"", ""3""]","""南葵中心3隊""",19,10,55591.414062,43091.414062,46675.914062,2456.626953,3
"[{""skc-4@example.com"",""南葵中心4隊"",""2024-08-02T20:22:21.223Z"",""2024-08-02T20:19:06.128Z"",1,""xFldNrcu765eRZc2hrsn"",""聖葉理諾堂""}, {""skc-4@example.com"",""南葵中心4隊"",""2024-08-02T21:41:43.053Z"",""2024-08-02T20:20:21.384Z"",1,""yhDNSAU3ByPksCfmW3E3"",""天河路遊樂場涼亭""}, … {""skc-4@example.com"",""南葵中心4隊"",""2024-08-02T22:33:31.782Z"",""2024-08-02T21:42:41.441Z"",3,""gFJqiYep8W1OwbFTueIM"",""天瑞路公園涼亭""}]","""skc-4@example.com""","[""4"", ""1""]","""南葵中心4隊""",26,10,44894.796875,32394.796875,77038.289062,2963.01123,3


In [14]:
fig = px.bar(
    users_df.sort('score_offset'), 
    x='name', 
    y='score', 
    hover_data=['name', 'score', 'total_captured', 'final_level'], 
    labels={
        'name': "隊伍",
        'score_offset': "分數",
        'score': "分數",
        'total_captured': "總佔領次數",
        'final_level': "最終等級"
    }, 
    color='name', 
    color_discrete_map=color_discrete_map, 
    text_auto='.0f',
    width=800,
)

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(title_text='隊伍分數排行(對Boss傷害) <br><sup>分數由佔領時間及等級計算</sup>', title_x=0.5)

fig.show()

In [15]:
fig = px.bar(
    users_df.sort('duration'), 
    x='name', 
    y='duration', 
    hover_data=['name', 'score', 'total_captured', 'duration', 'final_level'], 
    labels={
        'name': "隊伍",
        'score': "分數",
        'score_offset': "分數",
        'total_captured': "總佔領次數",
        'final_level': "最終等級",
        'duration': "佔領時長"
    }, 
    color='name', 
    color_discrete_map=color_discrete_map, 
    text_auto='.0f',
    width=800,
)

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(title_text='總佔領時長 <br><sup>隊伍所有佔領點合共的佔領時間</sup>', title_x=0.5)

fig.show()

In [16]:
fig = px.bar(
    users_df.sort('total_captured'), 
    x='name', 
    y='total_captured', 
    hover_data=['name', 'score', 'total_captured', 'duration', 'final_level', 'total_captured'], 
    labels={
        'name': "隊伍",
        'score': "分數",
        'score_offset': "分數",
        'total_captured': "總佔領次數",
        'final_level': "最終等級",
        'duration': "佔領時長",
        'total_captured': "總佔領次數"
    }, 
    color='name', 
    color_discrete_map=color_discrete_map, 
    text_auto='.0f',
    width=800,
)

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(title_text='總佔領次數 <br><sup>隊伍Scan了多少次QRCode</sup>', title_x=0.5)

fig.show()

In [17]:
fig = px.bar(
    users_df.sort('total_captured_unique'), 
    x='name', 
    y='total_captured_unique', 
    hover_data=['name', 'score', 'total_captured', 'duration', 'final_level', 'total_captured', 'total_captured_unique'], 
    labels={
        'name': "隊伍",
        'score': "分數",
        'score_offset': "分數",
        'total_captured': "總佔領次數",
        'final_level': "最終等級",
        'duration': "佔領時長",
        'total_captured': "總佔領次數",
        'total_captured_unique': "總到訪佔領點"
    }, 
    color='name', 
    color_discrete_map=color_discrete_map, 
    text_auto='.0f',
    width=800,
)

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(title_text='總到訪佔領點 <br><sup>隊伍到訪了多少個不同的佔領點</sup>', title_x=0.5)

fig.show()

In [18]:
users_df['score'].describe()

statistic,value
str,f64
"""count""",9.0
"""null_count""",0.0
"""mean""",17606.554688
"""std""",19412.121094
"""min""",0.0
"""25%""",6006.617188
"""50%""",10107.545898
"""75%""",15474.444336
"""max""",55591.414062


In [19]:
df = px.data.tips()
fig = px.box(
    users_df, 
    y="score",
    points="all",
    width=400,
    labels={
        'score': "分數",
    }
)

fig.update_layout(title_text='分數分布', title_x=0.5)

fig.show()

In [20]:
fig = px.scatter(
    users_df, 
    x="duration", 
    y="score",
    
    hover_data=['name', 'score', 'total_captured', 'duration', 'final_level'], 
    labels={
        'name': "隊伍",
        'score': "分數",
        'score_offset': "分數",
        'total_captured': "總佔領次數",
        'final_level': "最終等級",
        'duration': "佔領時長"
    }, 
    color='name', 
    color_discrete_map=color_discrete_map, 
    width=600,
    height=500
)


fig.update_layout(title_text='分數 和 佔領時長 的關係', title_x=0.5)

fig.show()

In [21]:
fig = px.scatter(
    users_df, 
    x="total_captured", 
    y="score",
    
    hover_data=['name', 'score', 'total_captured', 'duration', 'final_level'], 
    labels={
        'name': "隊伍",
        'score': "分數",
        'score_offset': "分數",
        'total_captured': "總佔領次數",
        'final_level': "最終等級",
        'duration': "佔領時長"
    }, 
    color='name', 
    color_discrete_map=color_discrete_map, 
    width=600,
    height=500
)


fig.update_layout(title_text='分數 和 佔領次數 的關係', title_x=0.5)

fig.show()

In [22]:
fig = px.density_mapbox(
    captured_logs_df.group_by('point_name').agg(pl.count()).join(points_df[['point', 'lat', 'long']], left_on="point_name", right_on="point"),
    lat='lat',
    lon='long',
    z='count',
    radius=200,
    center=dict(lat=22.45873, lon=114.00217),
    zoom=14,
    mapbox_style="open-street-map",
    
    width=600,
    height=800,
    opacity=0.6
)

point_fig = px.scatter_mapbox(points_df,
    lat="lat",
    lon='long',
    hover_name="point",
    mapbox_style="open-street-map",
    center=dict(lat=22.45873, lon=114.00217),
    zoom=14,
    
    width=600,
    height=800,
    opacity=0.75,
)

fig.add_trace(point_fig.data[0])

fig.update_layout(title_text='佔領次數集中位置', title_x=0.5)

fig.show()


/var/folders/jw/hzz2wp3s7tq_30qpr57f65w40000gn/T/ipykernel_4310/1183825878.py:2: DeprecationWarning:

`pl.count()` is deprecated. Please use `pl.len()` instead.



In [23]:
from datetime import datetime, timedelta
import pytz

user_list = []
for user in users_df[['name', 'capturedPoints']].to_dicts():
    _pts = user['capturedPoints']
    
    pts =  []
    for pt in _pts:
        pt['createdAt'] = (datetime.fromisoformat(pt['createdAt'].replace('Z', '+00:00'))).astimezone(pytz.timezone('Asia/Hong_Kong'))
        pt['expiredAt'] = (datetime.fromisoformat(pt['expiredAt'].replace('Z', '+00:00'))).astimezone(pytz.timezone('Asia/Hong_Kong'))
        
        pts.append(pt)
    user_list.append(user)
    
user_list

[{'name': '南葵中心3隊',
  'capturedPoints': [{'userId': 'skc-3@example.com',
    'userName': '南葵中心3隊',
    'expiredAt': datetime.datetime(2024, 8, 3, 5, 44, 31, 341000, tzinfo=<DstTzInfo 'Asia/Hong_Kong' HKT+8:00:00 STD>),
    'createdAt': datetime.datetime(2024, 8, 3, 4, 27, 48, 160000, tzinfo=<DstTzInfo 'Asia/Hong_Kong' HKT+8:00:00 STD>),
    'level': 2,
    'pointId': 'MM12lHGWCtvCbQvMy29H',
    'pointName': '天瑞俓（二）'},
   {'userId': 'skc-3@example.com',
    'userName': '南葵中心3隊',
    'expiredAt': datetime.datetime(2024, 8, 3, 5, 42, 11, 260000, tzinfo=<DstTzInfo 'Asia/Hong_Kong' HKT+8:00:00 STD>),
    'createdAt': datetime.datetime(2024, 8, 3, 4, 29, 51, 988000, tzinfo=<DstTzInfo 'Asia/Hong_Kong' HKT+8:00:00 STD>),
    'level': 2,
    'pointId': 'Y54hD80CffPsd7hG6KZn',
    'pointName': '天瑞俓（三）'},
   {'userId': 'skc-3@example.com',
    'userName': '南葵中心3隊',
    'expiredAt': datetime.datetime(2024, 8, 3, 5, 36, 41, 603000, tzinfo=<DstTzInfo 'Asia/Hong_Kong' HKT+8:00:00 STD>),
    'createdA

In [24]:
log_list = captured_logs_df.with_columns(pl.col("user_level").cast(pl.String)).with_columns(pl.col('datetime') + pl.duration(hours=8)).with_columns(pl.col('datetime'))[['datetime', 'user_name']].to_dicts()
log_list

[{'datetime': datetime.datetime(2024, 8, 3, 6, 37, 38, 56000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊'},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 35, 6, 707000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊'},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 26, 56, 297000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊'},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 22, 50, 933000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊'},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 15, 27, 703000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環/將軍澳中心隊'},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 14, 0, 116000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊'},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 12, 10, 551000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環/將軍澳中心隊'},
 {'datetime': datetime.datetime(2024

In [25]:
def map_score(data):
    score_per_second = {
        "_": 1 / 60,
        "1": 1 / 60,
        "2": 20 / 60,
        "3": 150 / 60,
        "4": 250 / 60,
        "5": 500 / 60,
    }
    
    total_score = 0
    for item in data:
        duration = (item['expiredAt'] - item['createdAt']).total_seconds()
        level = str(item['level'])
        score = duration * score_per_second.get(level, score_per_second["_"])
        total_score += score
    return total_score

for user in user_list:
    log_list.append({'datetime': datetime.fromisoformat("2024-08-03 06:27:48.056000+08:00"), 'user_name': user['name']})

log_with_score = []
for log in log_list:
    # Get Available user record
    user = [u for u in user_list if u['name'] == log['user_name']][0]
    capturedPoints = user['capturedPoints']
    
    # Filter time after log datetime
    capturedPoints = [{**cp, "expiredAt": cp['expiredAt'] if log['datetime'] > cp['expiredAt'] else log['datetime'] } for cp in capturedPoints if log['datetime'] > cp['createdAt']]
    
    score = map_score(capturedPoints)
    # print(log['datetime'], log['user_name'], len(capturedPoints), round(score, 2))
    
    log_with_score.append({
        "datetime": log['datetime'], 
        "user_name": log['user_name'], 
        "score": round(score, 2)
    })
log_with_score   

[{'datetime': datetime.datetime(2024, 8, 3, 6, 37, 38, 56000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊',
  'score': 6006.62},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 35, 6, 707000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊',
  'score': 6621.39},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 26, 56, 297000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊',
  'score': 3727.95},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 22, 50, 933000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊',
  'score': 2418.12},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 15, 27, 703000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環/將軍澳中心隊',
  'score': 7524.53},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 14, 0, 116000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')),
  'user_name': '上環中心隊',
  'score': 912.74},
 {'datetime': datetime.datetime(2024, 8, 3, 6, 12, 10, 551

In [26]:
log_with_score_df = pl.DataFrame(log_with_score).sort('datetime', descending=True).with_columns(pl.col('datetime') + pl.duration(hours=8))
log_with_score_df

datetime,user_name,score
"datetime[μs, UTC]",str,f64
2024-08-03 06:37:38.056 UTC,"""上環中心隊""",6006.62
2024-08-03 06:35:06.707 UTC,"""上環中心隊""",6621.39
2024-08-03 06:27:48.056 UTC,"""南葵中心3隊""",51294.84
2024-08-03 06:27:48.056 UTC,"""南葵中心4隊""",42202.28
2024-08-03 06:27:48.056 UTC,"""上環/將軍澳中心隊""",12953.79
…,…,…
2024-08-03 04:22:29.130 UTC,"""南葵中心1隊""",0.13
2024-08-03 04:20:21.639 UTC,"""南葵中心4隊""",1.26
2024-08-03 04:19:06.398 UTC,"""南葵中心4隊""",0.0


In [27]:
from plotly.subplots import make_subplots

fig = px.scatter(
    captured_logs_df.with_columns(pl.col("user_level").cast(pl.String)).with_columns(pl.col('datetime') + pl.duration(hours=8)).join(users_df['capturedPoints', 'name', 'score'], left_on="user_name", right_on="name").sort('score', descending=True),
    y="user_name", 
    x="datetime",     
    color='user_level', 
    color_discrete_map={
        '1': "#5f5f5f",
        '2': "#2dd55b",
        '3': "#0054e9",
    }, 
    labels={
        'user_name': "隊伍",
        'user_level': "佔領等級",
        'datetime': "時間"
    }, 
)

fig.update_layout(title_text='佔領攻擊點的時間', title_x=0.5)


point_fig = px.scatter(
    upgrade_logs_df.with_columns(pl.col('datetime') + pl.duration(hours=8)).join(users_df, left_on="user_name", right_on="name"),
    y="user_name", 
    x="datetime",
    color_discrete_sequence=['red'],
)
point_fig.update_traces(marker={'size': 10})

line_fig = px.line(
    log_with_score_df, 
    x='datetime', 
    y='score', 
    color='user_name',
    color_discrete_map=color_discrete_map,
    markers=True
)

combined_fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces from fig
for trace in fig.data:
    combined_fig.add_trace(trace, secondary_y=False)

# Add traces from point_fig
for trace in point_fig.data:
    combined_fig.add_trace(trace, secondary_y=False)

# Add traces from line_fig
for trace in line_fig.data:
    combined_fig.add_trace(trace, secondary_y=True)

# Update layout
combined_fig.update_layout(
    title_text='佔領攻擊點的 時間 與 分數走勢',
    title_x=0.5
)
combined_fig.update_traces(marker_size=10)
combined_fig.update_xaxes(nticks=30)

combined_fig.show()


In [28]:
fig = px.scatter(
    captured_logs_df.with_columns(pl.col("user_level").cast(pl.String)).with_columns(pl.col('datetime') + pl.duration(hours=8)).join(users_df['capturedPoints', 'name', 'score'], left_on="user_name", right_on="name").sort('score', descending=True),
    y="user_name", 
    x="datetime",     
    color='user_level', 
    color_discrete_map={
        '1': "#5f5f5f",
        '2': "#2dd55b",
        '3': "#0054e9",
    }, 
    labels={
        'user_name': "隊伍",
        'user_level': "佔領等級",
        'datetime': "時間"
    }, 
)
fig.update_traces(marker_size=10)
fig.update_xaxes(nticks=30)

fig.update_layout(title_text='佔領攻擊點的時間', title_x=0.5)


point_fig = px.scatter(
    upgrade_logs_df.with_columns(pl.col('datetime') + pl.duration(hours=8)).join(users_df, left_on="user_name", right_on="name"),
    y="user_name", 
    x="datetime",
    color_discrete_sequence=['red'],
)
point_fig.update_traces(marker={'size': 10})

fig.add_trace(point_fig.data[0])

fig

In [29]:
# Update layout
line_fig.update_layout(
    title_text='分數走勢',
    title_x=0.5
)

line_fig.show()

In [30]:
logs_with_duration = pl.DataFrame()

for team in captured_logs_df['user_name'].unique().sort().to_list():
    logs = captured_logs_df.sort('user_name').filter(pl.col('user_name') == team).sort('datetime', descending=True).to_dicts()
    results = []
    
    last = None
    
    total = 0
    for log in logs:
        if last is None:
            last = log['datetime']
            log['duration'] = None
        else:
            # print(team, last, log['datetime'], (last - log['datetime']).total_seconds() / 60)
            duration = (last - log['datetime']).total_seconds()
            last = log['datetime']
            log['duration'] = duration
        results.append(log)
    print(results)
    logs_with_duration = pl.concat([logs_with_duration, pl.DataFrame(results)])

logs_with_duration = logs_with_duration.sort('datetime', descending=True).with_columns(logs_with_duration['duration'] / 60)
logs_with_duration

[{'datetime': datetime.datetime(2024, 8, 2, 22, 15, 27, 703000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')), 'type': 'CAPTURE_POINT', 'user_email': 'sw-2@example.com', 'user_name': '上環/將軍澳中心隊', 'user_level': 3, 'point_id': 'rSnhF4HYEnTG98p9E7To', 'point_name': '天水圍公園（西）', 'point_user_email': 'sw-2@example.com', 'point_user_name': '上環/將軍澳中心隊', 'duration': None}, {'datetime': datetime.datetime(2024, 8, 2, 22, 12, 10, 551000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')), 'type': 'CAPTURE_POINT', 'user_email': 'sw-2@example.com', 'user_name': '上環/將軍澳中心隊', 'user_level': 3, 'point_id': '45ntnYbmnHBUIEyvTtNs', 'point_name': '天瑞社區中心', 'point_user_email': 'sw-2@example.com', 'point_user_name': '上環/將軍澳中心隊', 'duration': 197.152}, {'datetime': datetime.datetime(2024, 8, 2, 21, 46, 55, 834000, tzinfo=zoneinfo.ZoneInfo(key='Asia/Hong_Kong')), 'type': 'CAPTURE_POINT', 'user_email': 'sw-2@example.com', 'user_name': '上環/將軍澳中心隊', 'user_level': 2, 'point_id': 'zoKPwjKHPyiu0i6BtHmx', 'point_name': 

datetime,type,user_email,user_name,user_level,point_id,point_name,point_user_email,point_user_name,duration
"datetime[μs, UTC]",str,str,str,i64,str,str,str,str,f64
2024-08-02 14:37:38.056 UTC,"""CAPTURE_POINT""","""sw-1@example.com""","""上環中心隊""",3,"""0gHiSX9dMHkVpffFR9RU""","""天水圍體育館""","""sw-1@example.com""","""上環中心隊""",null
2024-08-02 14:35:06.707 UTC,"""CAPTURE_POINT""","""sw-1@example.com""","""上環中心隊""",3,"""Un4smPc0lSA8jpfvA5zJ""","""天水圍游泳池""","""sw-1@example.com""","""上環中心隊""",3.285867
2024-08-02 14:26:56.297 UTC,"""CAPTURE_POINT""","""sw-1@example.com""","""上環中心隊""",3,"""YvzEAhiBhJrOstmGXoH3""","""天榮路東""","""sw-1@example.com""","""上環中心隊""",25.245283
2024-08-02 14:22:50.933 UTC,"""CAPTURE_POINT""","""sw-1@example.com""","""上環中心隊""",3,"""IeOJr7CsyHXad2DUL3n4""","""栢慧豪廷門口""","""sw-1@example.com""","""上環中心隊""",2.650383
2024-08-02 14:15:27.703 UTC,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",3,"""rSnhF4HYEnTG98p9E7To""","""天水圍公園（西）""","""sw-2@example.com""","""上環/將軍澳中心隊""",15.944017
…,…,…,…,…,…,…,…,…,…
2024-08-02 12:22:29.130 UTC,"""CAPTURE_POINT""","""skc-1@example.com""","""南葵中心1隊""",1,"""xFldNrcu765eRZc2hrsn""","""聖葉理諾堂""","""skc-1@example.com""","""南葵中心1隊""",1.901833
2024-08-02 12:20:21.639 UTC,"""CAPTURE_POINT""","""skc-4@example.com""","""南葵中心4隊""",1,"""yhDNSAU3ByPksCfmW3E3""","""天河路遊樂場涼亭""","""skc-4@example.com""","""南葵中心4隊""",7.26445
2024-08-02 12:19:06.398 UTC,"""CAPTURE_POINT""","""skc-4@example.com""","""南葵中心4隊""",1,"""xFldNrcu765eRZc2hrsn""","""聖葉理諾堂""","""skc-4@example.com""","""南葵中心4隊""",3.508633


In [31]:
logs_with_duration.filter(pl.col('user_name') == "上環/將軍澳中心隊")

datetime,type,user_email,user_name,user_level,point_id,point_name,point_user_email,point_user_name,duration
"datetime[μs, UTC]",str,str,str,i64,str,str,str,str,f64
2024-08-02 14:15:27.703 UTC,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",3,"""rSnhF4HYEnTG98p9E7To""","""天水圍公園（西）""","""sw-2@example.com""","""上環/將軍澳中心隊""",15.944017
2024-08-02 14:12:10.551 UTC,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",3,"""45ntnYbmnHBUIEyvTtNs""","""天瑞社區中心""","""sw-2@example.com""","""上環/將軍澳中心隊""",4.096283
2024-08-02 13:46:55.834 UTC,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",2,"""zoKPwjKHPyiu0i6BtHmx""","""天業路公園（裏）""","""sw-2@example.com""","""上環/將軍澳中心隊""",5.175417
2024-08-02 13:44:16.811 UTC,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",2,"""kqK4XRQZ348HdJEaKjEd""","""天晴社區綜合服務大樓""","""sw-2@example.com""","""上環/將軍澳中心隊""",null
2024-08-02 13:28:20.170 UTC,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",2,"""4WvOcZ1x4sDSnEXete8m""","""天業路公園（表）""","""sw-2@example.com""","""上環/將軍澳中心隊""",2.894267
…,…,…,…,…,…,…,…,…,…
2024-08-02 13:15:07.887 UTC,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",2,"""ObC8rDBZIJcoRZJfBp1v""","""天悅天橋""","""sw-2@example.com""","""上環/將軍澳中心隊""",7.000833
2024-08-02 13:06:23.062 UTC,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",2,"""cWpn4JjGvqovfXr7L2rP""","""天城路天橋""","""sw-2@example.com""","""上環/將軍澳中心隊""",1.485917
2024-08-02 12:33:32.699 UTC,"""CAPTURE_POINT""","""sw-2@example.com""","""上環/將軍澳中心隊""",1,"""778jzmzKpOvgRHIHcHms""","""麗湖居""","""sw-2@example.com""","""上環/將軍澳中心隊""",18.006433


In [32]:
logs_with_duration.group_by('user_name').agg(pl.mean('duration')).sort('user_name')

user_name,duration
str,f64
"""上環/將軍澳中心隊""",7.412096
"""上環中心隊""",11.234442
"""南區中心1隊""",20.793233
"""南葵中心1隊""",4.176317
"""南葵中心2隊""",4.260762
"""南葵中心3隊""",2.891547
"""南葵中心4隊""",6.367928
"""石蔭中心隊""",3.822197


In [33]:
logs_with_duration['duration'].describe()

statistic,value
str,f64
"""count""",112.0
"""null_count""",8.0
"""mean""",5.391015
"""std""",5.742222
"""min""",0.08465
"""25%""",1.901833
"""50%""",3.8375
"""75%""",6.488983
"""max""",32.839383


In [34]:
fig = px.bar(
    logs_with_duration.group_by('user_name').agg(pl.mean('duration')).sort('duration'),
    x='user_name', 
    y='duration', 
    labels={
        'user_name': "隊伍",
        'duration': "佔領時長"
    }, 
    color='user_name', 
    color_discrete_map=color_discrete_map, 
    text_auto='.0f',
    width=800,
)

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(title_text='平均佔領攻擊點所需時間(分鐘) <br><sup>佔領攻擊點的時間差距</sup>', title_x=0.5)

fig.show()

In [35]:
fig = px.scatter(
    logs_with_duration.group_by('user_name').agg(pl.mean('duration')).sort('user_name').join(users_df, left_on='user_name', right_on="name"), 
    x="duration", 
    y="score",
    
    labels={
        'name': "隊伍",
        'score': "分數",
        'score_offset': "分數",
        'total_captured': "總佔領次數",
        'final_level': "最終等級",
        'duration': "佔領時長"
    }, 
    color='user_name', 
    color_discrete_map=color_discrete_map, 
    width=600,
    height=500
)


fig.update_layout(title_text='分數 和 佔領所需時間 的關係', title_x=0.5)

fig.show()

In [36]:
fig = px.bar(
    users_df[['name', 'average_duration']].with_columns(pl.col('average_duration') / 60).sort('average_duration'),
    x='name', 
    y='average_duration', 
    labels={
        'name': "隊伍",
        'average_duration': "時長"
    }, 
    color='name', 
    color_discrete_map=color_discrete_map, 
    text_auto='.0f',
    width=800,
)

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
fig.update_layout(title_text='平均持有攻擊點時間', title_x=0.5)

fig.show()

In [37]:
users_df[['name', 'average_duration']].with_columns(pl.col('average_duration') / 60).describe()

statistic,name,average_duration
str,str,f64
"""count""","""9""",9.0
"""null_count""","""0""",0.0
"""mean""",null,35.25642
"""std""",null,19.178276
"""min""","""上環/將軍澳中心隊""",0.0
"""25%""",null,29.394726
"""50%""",null,35.449959
"""75%""",null,49.383522
"""max""","""石蔭中心隊""",61.957169


In [38]:
user_captured_points_df = pl.DataFrame([y for x in users_df['capturedPoints'].to_list() for y in x])
map_data = user_captured_points_df.join(points_df, left_on="pointName", right_on="point")
map_data.sort('createdAt')


    # mapbox_zoom=14,
    # mapbox_center={"lat": 22.45873, "lon": 114.00217},

userId,userName,expiredAt,createdAt,level,pointId,pointName,isPublic,heroImage,long,lat
str,str,str,str,i64,str,str,bool,str,f64,f64
"""sy1@example.com""","""石蔭中心隊""","""2024-08-02T20:23:05.036Z""","""2024-08-02T20:14:55.939Z""",1,"""0gHiSX9dMHkVpffFR9RU""","""天水圍體育館""",true,"""https://firebasestorage.google…",114.00674,22.454822
"""sy1@example.com""","""石蔭中心隊""","""2024-08-02T20:27:41.262Z""","""2024-08-02T20:18:47.636Z""",1,"""Un4smPc0lSA8jpfvA5zJ""","""天水圍游泳池""",true,"""https://firebasestorage.google…",114.006509,22.455719
"""skc-4@example.com""","""南葵中心4隊""","""2024-08-02T20:22:21.223Z""","""2024-08-02T20:19:06.128Z""",1,"""xFldNrcu765eRZc2hrsn""","""聖葉理諾堂""",true,"""https://firebasestorage.google…",114.001623,22.44961
"""skc-4@example.com""","""南葵中心4隊""","""2024-08-02T21:41:43.053Z""","""2024-08-02T20:20:21.384Z""",1,"""yhDNSAU3ByPksCfmW3E3""","""天河路遊樂場涼亭""",true,"""https://firebasestorage.google…",114.000367,22.449815
"""skc-1@example.com""","""南葵中心1隊""","""2024-08-02T21:07:15.858Z""","""2024-08-02T20:22:21.223Z""",1,"""xFldNrcu765eRZc2hrsn""","""聖葉理諾堂""",true,"""https://firebasestorage.google…",114.001623,22.44961
…,…,…,…,…,…,…,…,…,…,…
"""sw-2@example.com""","""上環/將軍澳中心隊""","""2024-08-02T22:33:31.782Z""","""2024-08-02T22:15:26.468Z""",3,"""rSnhF4HYEnTG98p9E7To""","""天水圍公園（西）""",true,"""https://firebasestorage.google…",113.999815,22.455831
"""sw-1@example.com""","""上環中心隊""","""2024-08-02T22:33:31.782Z""","""2024-08-02T22:22:50.375Z""",3,"""IeOJr7CsyHXad2DUL3n4""","""栢慧豪廷門口""",true,"""https://firebasestorage.google…",114.001939,22.459445
"""sw-1@example.com""","""上環中心隊""","""2024-08-02T22:33:31.782Z""","""2024-08-02T22:26:55.806Z""",3,"""YvzEAhiBhJrOstmGXoH3""","""天榮路東""",true,"""https://firebasestorage.google…",114.001821,22.460974


In [40]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

color_discrete_map = {
    "南葵中心3隊": "#ea3323",
    "南葵中心4隊": "#ff8b00",
    "上環/將軍澳中心隊": "#febb26",
    "南葵中心2隊": "#17873f",
    "南葵中心1隊": "#25dc67",
    "南區中心1隊": "#4f7af9",
    "上環中心隊": "#9c78fe",
    "石蔭中心隊": "#ffccdd",
    "南區中心2隊": "#ffffff",
}

df = pl.DataFrame(map_data)

# Convert date columns to datetime
df = df.with_columns(
    pl.col("createdAt").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%fZ"),
    pl.col("expiredAt").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%fZ")
)

# Generate 5-minute intervals
start_time = df["createdAt"].min()
end_time = df["expiredAt"].max()

intervals = []
current_time = start_time

while current_time <= end_time:
    intervals.append(current_time)
    current_time += datetime.timedelta(minutes=3)

# Create a Plotly scatter mapbox plot
fig = go.Figure()

# Add scatter traces for each time step
for interval in intervals:
    filtered_df = df.filter(pl.col("createdAt") <= interval).filter(pl.col("expiredAt") >= interval)
    c = [color_discrete_map[x['userName']] for x in filtered_df.to_dicts()]
    fig.add_trace(go.Scattermapbox(
        lon=filtered_df["long"],
        lat=filtered_df["lat"],
        mode='markers',
        marker=go.scattermapbox.Marker(size=15, color=c),
        name=str(interval),
        text=filtered_df["userName"],
    ))
    
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}],
        label=intervals[i].strftime("%H:%M:%S")
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

# Update layout with mapbox style and initial view
fig.update_layout(
    # updatemenus=[{
    #     "direction": "left",
    #     "pad": {"r": 10, "t": 87},
    #     "showactive": False,
    #     "type": "buttons",
    #     "x": 0.1,
    #     "xanchor": "right",
    #     "y": 0,
    #     "yanchor": "top"
    # }],
    # sliders=[{
    #     "steps": [
    #         {"args": [[str(interval)], {"frame": {"duration": 300, "redraw": True}, "mode": "immediate"}],
    #          "label": str(interval),
    #          "method": "animate"} for interval in intervals
    #     ],
    #     "transition": {"duration": 300},
    #     "x": 0.1,
    #     "len": 0.9
    # }]
    
    sliders=[dict(
        steps=steps
    )]

)

fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=14,
    mapbox_center={"lat": 22.45873, "lon": 114.00217},
    width=800,
    height=1000,
)

fig.show()


/var/folders/jw/hzz2wp3s7tq_30qpr57f65w40000gn/T/ipykernel_4310/61009926.py:23: ChronoFormatWarning:

Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime

/var/folders/jw/hzz2wp3s7tq_30qpr57f65w40000gn/T/ipykernel_4310/61009926.py:24: ChronoFormatWarning:

Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime

